<a href="https://colab.research.google.com/github/Byeon-MJ/Computer_Vision_Project/blob/main/Predict_Handwritten_Digits_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# drive 마운트
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# 모듈 임포트
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# 이미지 파일 불러오기, 정보 확인
frame = '/content/gdrive/MyDrive/CV/Digits Classify/435.jpg'

img = cv2.imread(frame)
plt.imshow(img)
print('width: {} pixels'.format(img.shape[1]))
print('height: {} pixels'.format(img.shape[0]))
print('channels: {}'.format(img.shape[2]))

In [ ]:
# HSV Filter 적용하고 channel 분리
# 이미지를 BRG to HSV(HSV filter 적용)
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
hue, saturation, value = cv2.split(hsv)     # 색상(hue), 채도(saturation), 명도(value)
plt.imshow(value)                           # 명도 이미지를 보여준다

In [ ]:
# morphological operation 사용 위해 kernel 정의
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

In [ ]:
# topHat operation 적용
topHat = cv2.morphologyEx(value, cv2.MORPH_TOPHAT, kernel)
plt.imshow(topHat)

In [ ]:
# blackHat operation 적용
blackHat = cv2.morphologyEx(value, cv2.MORPH_BLACKHAT, kernel)
plt.imshow(blackHat)

In [ ]:
# add and subtract 수행
add = cv2.add(value, topHat)
subtract = cv2.subtract(add, blackHat)
plt.imshow(subtract)

In [ ]:
# gaussian blur 적용
blur = cv2.GaussianBlur(subtract, (5, 5), 0)
plt.imshow(blur)

In [ ]:
# blur 이미지에 threshold 적용
thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                               cv2.THRESH_BINARY_INV, 19, 9)

plt.imshow(thresh)

In [ ]:
# cv2.findContours 함수가 OpenCV 버전이 업그레이드 됨에 따라 파라미터 개수에 변동이 생겼음
# OpenCV 버전4 이상은 파라미터 개수가 2개, OpenCV 버전4 미만은 파라미터 개수가 3개

print(cv2.__version__)

cv2MajorVersion = cv2.__version__.split('.')[0]
print('openCV version : ', cv2MajorVersion)

# thresh 이미지에서 countour들을 찾아온다.
if int(cv2MajorVersion) >= 4:
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST,
                                             cv2.CHAIN_APPROX_SIMPLE)
else:
    imageContours, contours, hierarcy = cv2.findContours(thresh, cv2.RETR_LIST,
                                                         cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
img_digits = []     # digit 이미지가 저장되는 배열 정의
margin = 10         # digit 이미지 저장 여유 마진

for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)  # contour 주변을 사각형으로 찾아옴

    # 찾아온 이미지가 너무 작다면 무시
    if w * h < 100: continue

    # Y and X [y-margin : y + h + margin, x-margin:x+w+margin]
    # contour boxing 위치 좌표를 계산하여 thresh 이미지로부터 digit image를 구해 img_digits 배열에 넣어준다.
    img_digits.append(thresh[y-margin:y+h+margin, x-margin:x+w+margin])

    # 원본 이미지에 contour boxing 영역을 사각형으로 표시
    cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    print(x, x+w, y, y+h)

plt.imshow(img)

In [ ]:
# 저장된 digit image 확인
for i in range(len(img_digits)):
    plt.subplot(1, len(img_digits), i+1)
    plt.imshow(img_digits[i], cmap='Greys', interpolation='nearest')
    print(img_digits[i].shape)

plt.tight_layout()
plt.show()

In [ ]:
# input 위해 image size 재조정(28 x 28)
SZ = 28

for i in range(len(img_digits)):
    plt.subplot(1, len(img_digits), i+1)
    plt.imshow(cv2.resize(img_digits[i], (SZ, SZ)), cmap='Greys', interpolation='nearest')

plt.tight_layout()
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import models

# 모델 불러오기
model = models.load_model('/content/gdrive/MyDrive/CV/Digits Classify/digits_model.h5')
model.summary()

In [ ]:
# 분류 예측
SZ = 28

for i in range(len(img_digits)):
    plt.subplot(1, len(img_digits), i+1)
    
    # img_digits 배열의 크기가 서로 다른 이미지들을 모두 28X28 로 재조정
    num = cv2.resize(img_digits[i], (SZ, SZ))

    # 픽셀 값을 0~1 범위로 변환
    num = num.astype('float32') / 255.

    # 이미지 Display
    plt.imshow(num, cmap='Greys', interpolation='nearest')

    # 이미지를 우리가 만든 모델을 통해 분류 예측
    result = model.predict(np.array([num]))
    result_number = np.argmax(result)
    plt.title(result_number)
    performance = [val for val in result[0]]
    print(performance)

plt.tight_layout()
plt.show()